# MIMIC 4 data - dataset construction inputevents

Code taken from GRU-ODE-Bayes preprocessing; simplified and adapted for MIMIC 4 1.0

In [1]:
import os
import pathlib

p_project = str(pathlib.Path(os.getcwd()).parents[1])

In [2]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np

In [3]:
path_data = p_project + '/data/original/mimic4'
path_temp = p_project + '/data/mimic4'

In [4]:
adm_3 = pd.read_csv(path_temp + '/processed/tables/admissions_processed.csv')

In [5]:
# only choose previously selected admission ids
inputs=pd.read_csv(path_data + '/icu/inputevents.csv.gz')
adm_ids=list(adm_3['hadm_id'])
inputs=inputs.loc[inputs['hadm_id'].isin(adm_ids)]
inputs.head()

,subject_id,hadm_id,stay_id,starttime,endtime,storetime,itemid,amount,amountuom,rate,...,ordercategorydescription,patientweight,totalamount,totalamountuom,isopenbag,continueinnextdept,cancelreason,statusdescription,originalamount,originalrate
0,12481680,26876606,30863119,2141-01-23 17:37:00,2141-01-23 17:38:00,2141-01-23 17:37:00,226452,180.0,ml,NaN,...,Bolus,123.0,180.0,ml,0,0,0,FinishedRunning,180.0,180.0
1,12481680,26876606,30863119,2141-01-23 13:00:00,2141-01-23 13:01:00,2141-01-23 13:23:00,226452,240.0,ml,NaN,...,Bolus,123.0,240.0,ml,0,0,0,FinishedRunning,240.0,240.0
2,12481680,26876606,30863119,2141-01-23 11:00:00,2141-01-23 11:01:00,2141-01-23 12:24:00,226452,120.0,ml,NaN,...,Bolus,123.0,120.0,ml,0,0,0,FinishedRunning,120.0,120.0
3,12481680,26876606,30863119,2141-01-23 14:16:00,2141-01-23 14:17:00,2141-01-23 14:17:00,226452,60.0,ml,NaN,...,Bolus,123.0,60.0,ml,0,0,0,FinishedRunning,60.0,60.0
4,15614172,27424463,33484414,2153-07-30 23:16:00,2153-07-30 23:17:00,2153-07-30 23:16:00,226452,100.0,ml,NaN,...,Bolus,71.6,100.0,ml,0,0,0,FinishedRunning,100.0,100.0


In [6]:
# only keep columns of interest
inputs_small=inputs[['subject_id','hadm_id','starttime','endtime','itemid','amount','amountuom','rate','rateuom','patientweight','ordercategorydescription']]
print('Number of patients remaining in the database: ')
print(inputs_small['subject_id'].nunique())

Number of patients remaining in the database: 
44035


In [7]:
# get item ids for inputs 
item_id=pd.read_csv(path_data + '/icu/d_items.csv.gz')
item_id_1=item_id[['itemid','label']]
item_id_1.head()

inputs_small_2=pd.merge(inputs_small,item_id_1,on='itemid')
inputs_small_2.head()
print('Number of patients remaining in the database: ')
print(inputs_small_2['subject_id'].nunique())

Number of patients remaining in the database: 
44035


In [8]:
#Only select specific labels for the inputs.
#list of retained inputs :
retained_list=["Albumin 5%","Dextrose 5%","Lorazepam (Ativan)","Calcium Gluconate","Midazolam (Versed)","Phenylephrine","Furosemide (Lasix)","Norepinephrine","Magnesium Sulfate","Nitroglycerin","Insulin - Glargine","Insulin - Humalog","Insulin - Regular","Heparin Sodium","Morphine Sulfate","Potassium Chloride","Packed Red Blood Cells","Gastric Meds","D5 1/2NS","LR","K Phos","Solution","Sterile Water","Metoprolol","Piggyback","OR Crystalloid Intake","OR Cell Saver Intake","PO Intake","GT Flush","KCL (Bolus)","Magnesium Sulfate (Bolus)"]
#missing :Fresh Frozen Plasma
inputs_small_3=inputs_small_2.loc[inputs_small_2["label"].isin(retained_list)].copy()
print(inputs_small_3['label'].unique())

['PO Intake' 'Packed Red Blood Cells' 'Solution' 'Norepinephrine'
 'Gastric Meds' 'Dextrose 5%' 'GT Flush' 'Insulin - Glargine' 'Piggyback'
 'Potassium Chloride' 'K Phos' 'Insulin - Regular' 'Magnesium Sulfate'
 'LR' 'Metoprolol' 'Sterile Water' 'Insulin - Humalog' 'Heparin Sodium'
 'Furosemide (Lasix)' 'Morphine Sulfate' 'Midazolam (Versed)'
 'Lorazepam (Ativan)' 'Calcium Gluconate' 'D5 1/2NS'
 'OR Crystalloid Intake' 'Phenylephrine' 'Nitroglycerin'
 'Magnesium Sulfate (Bolus)' 'KCL (Bolus)' 'OR Cell Saver Intake'
 'Albumin 5%']


In [9]:
# print all content and check the commonly used metrics of each measurement
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(inputs_small_3.groupby('label')['amountuom'].value_counts())

label                      amountuom  
Albumin 5%                 ml              25504
Calcium Gluconate          grams           59358
D5 1/2NS                   ml              26609
                           L                   3
Dextrose 5%                ml             991362
Furosemide (Lasix)         mg              97376
GT Flush                   ml             206993
                           cm3                 1
Gastric Meds               ml             279300
                           L                   1
Heparin Sodium             units           72267
Insulin - Glargine         units           28935
Insulin - Humalog          units           82007
Insulin - Regular          units          274198
K Phos                     mmol             8666
KCL (Bolus)                ml              97436
LR                         ml             104922
                           L                   2
Lorazepam (Ativan)         mg              35637
Magnesium Sulfate          gra

In [10]:
##Cleaning the Heparin Sodium (Prophylaxis) (remove the non dose)
#inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='Heparin Sodium (Prophylaxis)') & (inputs_small_3['amountuom']!='dose')].index).copy()

#Cleaning the Magnesium Sulfate (remove the non grams)
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='Magnesium Sulfate') & (inputs_small_3['amountuom']!='grams')].index).copy()

#Cleaning the Metoprolol (remove the non mg)
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='Metoprolol') & (inputs_small_3['amountuom']!='mg')].index).copy()

#Cleaning the D5 1/2NS (keep ml)
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='D5 1/2NS') & (inputs_small_3['amountuom']!='ml')].index).copy()

#Cleaning the LR
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='LR') & (inputs_small_3['amountuom']!='ml')].index).copy()

#Cleaning the OR Crystalloid Intake 
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='OR Crystalloid Intake') & (inputs_small_3['amountuom']!='ml')].index).copy()

#Cleaning the PO Intake
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='PO Intake') & (inputs_small_3['amountuom']!='ml')].index).copy()

#Cleaning the Gastric Meds
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='Gastric Meds') & (inputs_small_3['amountuom']!='ml')].index).copy()

#Cleaning the GT Flush
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='GT Flush') & (inputs_small_3['amountuom']!='ml')].index).copy()

#Cleaning the Potassium Chloride
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='Potassium Chloride') & (inputs_small_3['amountuom']!='mEq')].index).copy()

In [11]:
# Verify
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(inputs_small_3.groupby('label')['amountuom'].value_counts())

label                      amountuom
Albumin 5%                 ml            25504
Calcium Gluconate          grams         59358
D5 1/2NS                   ml            26609
Dextrose 5%                ml           991362
Furosemide (Lasix)         mg            97376
GT Flush                   ml           206993
Gastric Meds               ml           279300
Heparin Sodium             units         72267
Insulin - Glargine         units         28935
Insulin - Humalog          units         82007
Insulin - Regular          units        274198
K Phos                     mmol           8666
KCL (Bolus)                ml            97436
LR                         ml           104922
Lorazepam (Ativan)         mg            35637
Magnesium Sulfate          grams         69094
Magnesium Sulfate (Bolus)  ml            49409
Metoprolol                 mg            57590
Midazolam (Versed)         mg           100572
Morphine Sulfate           mg            72149
Nitroglycerin          

In [12]:
# same thing for inputs given in rates
inputs_small_3.groupby('label')['rateuom'].value_counts()

label                      rateuom      
Albumin 5%                 mL/hour           25504
D5 1/2NS                   mL/hour           26398
Dextrose 5%                mL/hour          604226
                           mL/min                2
                           mL/kg/hour            1
Furosemide (Lasix)         mg/hour           18884
Heparin Sodium             units/hour        64621
                           units/kg/hour         2
Insulin - Regular          units/hour       132639
KCL (Bolus)                mL/hour           97436
LR                         mL/hour           56883
Lorazepam (Ativan)         mg/hour             181
Magnesium Sulfate (Bolus)  mL/hour           49407
                           mL/min                2
Midazolam (Versed)         mg/hour           47557
Morphine Sulfate           mg/hour            6773
Nitroglycerin              mcg/kg/min        76356
Norepinephrine             mcg/kg/min       320132
                           mg/kg/min     

In [13]:
#Cleaning of Dextrose 5%  (remove the non mL/hour)
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='Dextrose 5%') & (inputs_small_3['rateuom']!='mL/hour')].index).copy()

#Cleaning of Magnesium Sulfate (Bolus)  (remove the non mL/hour)
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='Magnesium Sulfate (Bolus)') & (inputs_small_3['rateuom']!='mL/hour')].index).copy()

#Cleaning of Piggyback (remove the non mL/hour)
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='Piggyback') & (inputs_small_3['rateuom']!='mL/hour')].index).copy()

#Cleaning of Packed Red Bllod Cells (remove the non mL/hour)
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='Packed Red Blood Cells') & (inputs_small_3['rateuom']!='mL/hour')].index).copy()

#Cleaning of Phenylephrine
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='Phenylephrine') & (inputs_small_3['rateuom']!='mcg/kg/min')].index).copy()

#Cleaning of Sterile Water 
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='Sterile Water') & (inputs_small_3['rateuom']!='mL/hour')].index).copy()

#Cleaning of Heparin Sodium 
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='Heparin Sodium') & (inputs_small_3['rateuom']!='units/hour')].index).copy()

#Cleaning of Norepinephrine
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='Norepinephrine') & (inputs_small_3['rateuom']!='mcg/kg/min')].index).copy()

#Cleaning of Solution
inputs_small_3=inputs_small_3.drop(inputs_small_3.loc[(inputs_small_3['label']=='Solution') & (inputs_small_3['rateuom']!='mL/hour')].index).copy()

#Check if a single unit per drug
inputs_small_3.groupby('label')['rateuom'].value_counts()

label                      rateuom   
Albumin 5%                 mL/hour        25504
D5 1/2NS                   mL/hour        26398
Dextrose 5%                mL/hour       604226
Furosemide (Lasix)         mg/hour        18884
Heparin Sodium             units/hour     64621
Insulin - Regular          units/hour    132639
KCL (Bolus)                mL/hour        97436
LR                         mL/hour        56883
Lorazepam (Ativan)         mg/hour          181
Magnesium Sulfate (Bolus)  mL/hour        49407
Midazolam (Versed)         mg/hour        47557
Morphine Sulfate           mg/hour         6773
Nitroglycerin              mcg/kg/min     76356
Norepinephrine             mcg/kg/min    320132
Packed Red Blood Cells     mL/hour        50446
Phenylephrine              mcg/kg/min    186434
Piggyback                  mL/hour        89844
Solution                   mL/hour       454218
Sterile Water              mL/hour        61400
Name: rateuom, dtype: int64

We now split the entries which are spread in time.
We chose the duration window for the sampling. here we choose 30 minutes. So every entry which has a rate and with duration larger than 1 hour, we split it into fixed times injections.

In [14]:
# We now split the entries which are spread in time. We chose the duration window for the sampling. here we choose 30 minutes. 
# So every entry which has a rate and with duration larger than 1 hour, we split it into fixed times injections.

#First check the /hours units
df_temp=inputs_small_3.loc[(inputs_small_3['rate'].notnull()) & (inputs_small_3['rateuom'].str.contains('mcg/kg/hour'))].copy()
df_temp['computed_amount']=df_temp['rate']*((pd.to_datetime(df_temp['endtime'])-pd.to_datetime(df_temp['starttime'])).dt.total_seconds()/3600)*df_temp['patientweight']

#Check with a 0.01 tolerance
assert(len(df_temp.loc[(abs(df_temp['computed_amount']-1000*df_temp['amount'])>0.01)].index)==0) #OK

df_temp=inputs_small_3.loc[(inputs_small_3['rate'].notnull()) & (inputs_small_3['rateuom'].str.contains('mL/hour'))].copy()
df_temp['computed_amount']=df_temp['rate']*((pd.to_datetime(df_temp['endtime'])-pd.to_datetime(df_temp['starttime'])).dt.total_seconds()/3600)

#Check with a 0.01 tolerance
assert(len(df_temp.loc[(abs(df_temp['computed_amount']-df_temp['amount'])>0.01)].index)==0) #OK

df_temp=inputs_small_3.loc[(inputs_small_3['rate'].notnull()) & (inputs_small_3['rateuom'].str.contains('mg/hour'))].copy()
df_temp['computed_amount']=df_temp['rate']*((pd.to_datetime(df_temp['endtime'])-pd.to_datetime(df_temp['starttime'])).dt.total_seconds()/3600)

#Check with a 0.01 tolerance
assert(len(df_temp.loc[(abs(df_temp['computed_amount']-df_temp['amount'])>0.01)].index)==0) #OK

df_temp=inputs_small_3.loc[(inputs_small_3['rate'].notnull()) & (inputs_small_3['rateuom'].str.contains('mcg/hour'))].copy()
df_temp['computed_amount']=df_temp['rate']*((pd.to_datetime(df_temp['endtime'])-pd.to_datetime(df_temp['starttime'])).dt.total_seconds()/3600)

#Check with a 0.01 tolerance
assert(len(df_temp.loc[(abs(df_temp['computed_amount']-df_temp['amount'])>0.01)].index)==0) #OK

df_temp=inputs_small_3.loc[(inputs_small_3['rate'].notnull()) & (inputs_small_3['rateuom'].str.contains('units/hour'))].copy()
df_temp['computed_amount']=df_temp['rate']*((pd.to_datetime(df_temp['endtime'])-pd.to_datetime(df_temp['starttime'])).dt.total_seconds()/3600)

#Check with a 0.01 tolerance
assert(len(df_temp.loc[(abs(df_temp['computed_amount']-df_temp['amount'])>0.01)].index)==0) #OK

df_temp=inputs_small_3.loc[(inputs_small_3['rate'].notnull()) & (inputs_small_3['rateuom'].str.contains('mg/min'))].copy()
df_temp['computed_amount']=df_temp['rate']*((pd.to_datetime(df_temp['endtime'])-pd.to_datetime(df_temp['starttime'])).dt.total_seconds()/60)

#Check with a 0.01 tolerance
assert(len(df_temp.loc[(abs(df_temp['computed_amount']-df_temp['amount'])>0.01)].index)==0) #OK

#Third check the kg/min units
df_temp=inputs_small_3.loc[(inputs_small_3['rate'].notnull()) & (inputs_small_3['rateuom'].str.contains('mcg/kg/min'))].copy()
df_temp['computed_amount']=df_temp['rate']*((pd.to_datetime(df_temp['endtime'])-pd.to_datetime(df_temp['starttime'])).dt.total_seconds()/60)*df_temp['patientweight']

#Check with a 0.01 tolerance
assert(len(df_temp.loc[(abs(df_temp['computed_amount']/1000-df_temp['amount'])>0.01)].index)==0) #OK

In [15]:
duration_split_hours=0.5
to_sec_fact=3600*duration_split_hours

#split data set in four.

#The first dataframe contains the entries with no rate but with extended duration inputs (over 0.5 hour)
df_temp1=inputs_small_3.loc[((pd.to_datetime(inputs_small_3['endtime'])-pd.to_datetime(inputs_small_3['starttime']))>timedelta(hours=duration_split_hours)) & (inputs_small_3['rate'].isnull())].copy().reset_index(drop=True)
#The second dataframe contains the entries with no rate and low duration entries (<0.5hour)
df_temp2=inputs_small_3.loc[((pd.to_datetime(inputs_small_3['endtime'])-pd.to_datetime(inputs_small_3['starttime']))<=timedelta(hours=duration_split_hours)) & (inputs_small_3['rate'].isnull())].copy().reset_index(drop=True)
#The third dataframe contains the entries with a rate and extended duration inputs (over 0.5 hour)
df_temp3=inputs_small_3.loc[((pd.to_datetime(inputs_small_3['endtime'])-pd.to_datetime(inputs_small_3['starttime']))>timedelta(hours=duration_split_hours)) & (inputs_small_3['rate'].notnull())].copy().reset_index(drop=True)
#The forth dataframe contains the entries with a rate and low duration entries (< 0.5 hour)
df_temp4=inputs_small_3.loc[((pd.to_datetime(inputs_small_3['endtime'])-pd.to_datetime(inputs_small_3['starttime']))<=timedelta(hours=duration_split_hours)) & (inputs_small_3['rate'].notnull())].copy().reset_index(drop=True)

#Check if split is complete
assert(len(df_temp1.index)+len(df_temp2.index)+len(df_temp3.index)+len(df_temp4.index)==len(inputs_small_3.index))

In [16]:
#We then process all of these dfs.
#In the first one, we need to duplicate the entries according to their duration and then divide each entry by the number of duplicates

#We duplicate the rows with the number bins for each injection
df_temp1['Repeat']=np.ceil((pd.to_datetime(df_temp1['endtime'])-pd.to_datetime(df_temp1['starttime'])).dt.total_seconds()/to_sec_fact).astype(int)
df_new1=df_temp1.reindex(df_temp1.index.repeat(df_temp1['Repeat']))

#We then create the admninistration time as a shifted version of the STARTTIME.
df_new1['charttime']=df_new1.groupby(level=0)['starttime'].transform(lambda x: pd.date_range(start=x.iat[0],freq=str(60*duration_split_hours)+'min',periods=len(x)))
#We divide each entry by the number of repeats
df_new1['amount']=df_new1['amount']/df_new1['Repeat']

In [17]:
# In the third one, we do the same
#We duplicate the rows with the number bins for each injection
df_temp3['Repeat']=np.ceil((pd.to_datetime(df_temp3['endtime'])-pd.to_datetime(df_temp3['starttime'])).dt.total_seconds()/to_sec_fact).astype(int)
df_new3=df_temp3.reindex(df_temp3.index.repeat(df_temp3['Repeat']))
#We then create the admninistration time as a shifted version of the STARTTIME.
df_new3['charttime']=df_new3.groupby(level=0)['starttime'].transform(lambda x: pd.date_range(start=x.iat[0],freq=str(60*duration_split_hours)+'min',periods=len(x)))
#We divide each entry by the number of repeats
df_new3['amount']=df_new3['amount']/df_new3['Repeat']

In [18]:
df_temp2['charttime']=df_temp2['starttime']
df_temp4['charttime']=df_temp4['starttime']

In [19]:
#Eventually, we merge all 4splits into one.
inputs_small_4=df_new1.append([df_temp2,df_new3,df_temp4], sort=True)
#The result is a dataset with discrete inputs for each treatment.
inputs_small_4.shape

(18593793, 14)

In [20]:
inputs_small_4.to_csv(path_temp + '/processed/tables/inputs_processed.csv')
inputs_small_4['hadm_id'].nunique()

55024

In [21]:
inputs_small_4.head(40)

,Repeat,amount,amountuom,charttime,endtime,hadm_id,itemid,label,ordercategorydescription,patientweight,rate,rateuom,starttime,subject_id
0,2.0,10.0,mEq,2171-03-02 01:22:00,2171-03-02 02:22:00,22367569,225166,Potassium Chloride,Continuous IV,94.1,NaN,NaN,2171-03-02 01:22:00,13441205
0,2.0,10.0,mEq,2171-03-02 01:52:00,2171-03-02 02:22:00,22367569,225166,Potassium Chloride,Continuous IV,94.1,NaN,NaN,2171-03-02 01:22:00,13441205
1,2.0,10.0,mEq,2171-02-28 00:00:00,2171-02-28 01:00:00,22367569,225166,Potassium Chloride,Continuous IV,94.1,NaN,NaN,2171-02-28 00:00:00,13441205
1,2.0,10.0,mEq,2171-02-28 00:30:00,2171-02-28 01:00:00,22367569,225166,Potassium Chloride,Continuous IV,94.1,NaN,NaN,2171-02-28 00:00:00,13441205
2,2.0,10.0,mEq,2171-02-28 01:00:00,2171-02-28 02:00:00,22367569,225166,Potassium Chloride,Continuous IV,94.1,NaN,NaN,2171-02-28 01:00:00,13441205
2,2.0,10.0,mEq,2171-02-28 01:30:00,2171-02-28 02:00:00,22367569,225166,Potassium Chloride,Continuous IV,94.1,NaN,NaN,2171-02-28 01:00:00,13441205
3,2.0,10.0,mEq,2171-03-01 14:32:00,2171-03-01 15:32:00,22367569,225166,Potassium Chloride,Continuous IV,94.1,NaN,NaN,2171-03-01 14:32:00,13441205
3,2.0,10.0,mEq,2171-03-01 15:02:00,2171-03-01 15:32:00,22367569,225166,Potassium Chloride,Continuous IV,94.1,NaN,NaN,2171-03-01 14:32:00,13441205
4,2.0,10.0,mEq,2171-03-02 02:35:00,2171-03-02 03:35:00,22367569,225166,Potassium Chloride,Continuous IV,94.1,NaN,NaN,2171-03-02 02:35:00,13441205
4,2.0,10.0,mEq,2171-03-02 03:05:00,2171-03-02 03:35:00,22367569,225166,Potassium Chloride,Continuous IV,94.1,NaN,NaN,2171-03-02 02:35:00,13441205
